In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [6]:
import json
import itertools

with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

pairs = []

for intent in data["intents"]:
    patterns = intent["patterns"]
    responses = intent["responses"]
    # Chọn 1 response ngẫu nhiên cho mỗi pattern (cũng có thể kết hợp tất cả)
    for p in patterns:
        for r in responses:  # hoặc chỉ r = random.choice(responses)
            pairs.append((p, r))

print(f"Total pairs: {len(pairs)}")
print(pairs[:10])


Total pairs: 528
[('Hi', 'Hello!'), ('Hi', 'Good to see you again!'), ('Hi', 'Hi there, how can I help?'), ('How are you?', 'Hello!'), ('How are you?', 'Good to see you again!'), ('How are you?', 'Hi there, how can I help?'), ('Is anyone there?', 'Hello!'), ('Is anyone there?', 'Good to see you again!'), ('Is anyone there?', 'Hi there, how can I help?'), ('Hello', 'Hello!')]


In [19]:
# ===============================
# 1. Dataset mini (câu hỏi → câu trả lời)
# ===============================

import re
words = []
print(len(pairs))
for q, a in pairs:
    q_clean = re.sub(r'[^\w\s]', '', q.lower())
    a_clean = re.sub(r'[^\w\s]', '', a.lower())
    words.append((q_clean, a_clean))
# Tạo từ điển char → index và index → char
all_text = " ".join([q +" "+ a for q, a in words])
print(all_text)
lst=list(set(all_text.split(" ")))
lst.extend(['\t','\n'])
chars = sorted(lst)
print(chars)
char2idx = {c:i for i,c in enumerate(chars)}
idx2char = {i:c for i,c in enumerate(chars)}

num_chars = len(chars)
max_len_input = max(len(q_clean.split(" ")) for q_clean, _ in words)
max_len_target = max(len(a_clean.split(" ")) for _, a_clean in words)
print(max_len_input)
print(max_len_target)

528
hi hello hi good to see you again hi hi there how can i help how are you hello how are you good to see you again how are you hi there how can i help is anyone there hello is anyone there good to see you again is anyone there hi there how can i help hello hello hello good to see you again hello hi there how can i help good day hello good day good to see you again good day hi there how can i help whats up hello whats up good to see you again whats up hi there how can i help how are ya hello how are ya good to see you again how are ya hi there how can i help heyy hello heyy good to see you again heyy hi there how can i help whatsup hello whatsup good to see you again whatsup hi there how can i help    hello    good to see you again    hi there how can i help cya sad to see you go  cya talk to you later cya goodbye cya come back soon see you sad to see you go  see you talk to you later see you goodbye see you come back soon bye bye sad to see you go  bye bye talk to you later bye bye g

In [8]:
# ===============================
# 2. Prepare input/output data (one-hot encoding)
# ===============================
def encode_text(text, max_len):
    x = np.zeros((1, max_len, num_chars), dtype='float32')
    clean_text= re.sub(r'[^\w\s]', '', text.lower())
    for t, char in enumerate(list(clean_text.split(" "))):
        x[0, t, char2idx[char]] = 1.0
    return x

encoder_input_data = []
decoder_input_data = []
decoder_target_data = []

for q, a in pairs:
    # Add start and end tokens for decoder
    a_in = "\t " + a  # start token
    a_out = a + " \n" # end token
    
    enc = encode_text(q, max_len_input)
    dec_in = encode_text(a_in, max_len_target + 1)
    dec_out = encode_text(a_out, max_len_target + 1)
    
    encoder_input_data.append(enc)
    decoder_input_data.append(dec_in)
    decoder_target_data.append(dec_out)

encoder_input_data = np.vstack(encoder_input_data)
decoder_input_data = np.vstack(decoder_input_data)
decoder_target_data = np.vstack(decoder_target_data)

In [9]:
# ===============================
# 3. Build Seq2Seq model
# ===============================
latent_dim = 16
#input
encoder_inputs = Input(shape=(None, num_chars))
decoder_inputs = Input(shape=(None, num_chars))


# Encoder
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_chars, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')

2025-09-12 15:42:58.923265: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-09-12 15:42:58.923299: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-09-12 15:42:58.923308: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2025-09-12 15:42:58.923346: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-12 15:42:58.923365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [25]:
# ===============================
# 4. Train model (rất nhanh vì dataset nhỏ)
# ===============================
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=1, epochs=100, verbose=1)

Epoch 1/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2686
Epoch 2/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2581
Epoch 3/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2568
Epoch 4/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2574
Epoch 5/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2655
Epoch 6/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2520
Epoch 7/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2551
Epoch 8/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2517
Epoch 9/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2517
Epoch 10/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2581
Epoch 11/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2750
Epoch 12/100
528/528 [==============================] - 5s 9ms/step - loss: 0.2674
Epoch 13/100


In [26]:





# ===============================
# 5. Inference (decode)
# ===============================
# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states_inf = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs_inf] + decoder_states_inf)

# Hàm giải mã
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, num_chars))
    target_seq[0, 0, char2idx["\t"]] = 1.0
    
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2char[sampled_token_index]
        
        if sampled_char == "\n" or len(decoded_sentence.split(" ")) >= max_len_target-1:
            stop_condition = True
            decoded_sentence += sampled_char
        else:
            decoded_sentence += sampled_char+" "
        
        target_seq = np.zeros((1, 1, num_chars))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    
    return decoded_sentence.strip()

encoder_model.save("encoder_model.h5")
decoder_model.save("decoder_model.h5")


In [27]:
# ===============================
# 6. Demo chatbot
# ===============================
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")
        break
    seq = encode_text(user_input, max_len_input)
    response = decode_sequence(seq)
    print("Chatbot:", response)

1/1 [==============================] - 0s 9ms/step
Chatbot: hi there how can i menu
1/1 [==============================] - 0s 9ms/step
Chatbot: good to see you again
Chatbot: Goodbye!


In [ ]:
import os

# Tạo thư mục assets nếu chưa có
os.makedirs("assets", exist_ok=True)

# Lưu char2idx và idx2char
with open("assets/char2idx.json", "w", encoding="utf-8") as f:
    json.dump(char2idx, f, ensure_ascii=False, indent=2)

# Lưu idx2char (chuyển key thành string để JSON đọc được)
idx2char_str = {str(k): v for k, v in idx2char.items()}
with open("assets/idx2char.json", "w", encoding="utf-8") as f:
    json.dump(idx2char_str, f, ensure_ascii=False, indent=2)

# Lưu config (số chiều, max_len…)
config = {
    "num_chars": num_chars,
    "max_len_input": max_len_input,
    "max_len_target": max_len_target,
    "latent_dim": latent_dim
}
with open("assets/config.json", "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print("Saved assets to assets/")


✅ Saved assets to assets/
